In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')

# Cropped Decoding

Now we will use cropped decoding. Cropped decoding means the ConvNet is trained on time windows/time crops within the trials. We will explain this visually by comparing trialwise to cropped decoding.



Trialwise Decoding | Cropped Decoding
- | - 
![Trialwise Decoding](./trialwise_explanation.png "Trialwise Decoding") | ![Cropped Decoding](./cropped_explanation.png "Cropped Decoding")

On the left, you see trialwise decoding:

1. A complete trial is pushed through the network
2. The network produces a prediction
3. The prediction is compared to the target (label) for that trial to compute the loss


On the right, you see cropped decoding:

1. Instead of a complete trial, windows within the trial, here called *crops*, are pushed through the network
2. For computational efficiency, multiple neighbouring crops are pushed through the network simultaneously (these neighbouring crops are called a *supercrop*)
3. Therefore, the network produces multiple predictions (one per crop in the supercrop)
4. The individual crop predictions are averaged before computing the loss function


Notes:

* The network architecture implicitly defines the crop size (it is the receptive field size, i.e., the number of timesteps the network uses to make a single prediction)
* The supercrop size is a user-defined hyperparameter, called `input_time_length` in Braindecode. It mostly affects runtime (larger supercrop sizes should be faster). As a rule of thumb, you can set it to two times the crop size.
* Crop size and supercrop size together define how many predictions the network makes per supercrop:  $\mathrm{\#supercrop}-\mathrm{\#crop}+1=\mathrm{\#predictions}$

For cropped decoding, the above training setup is mathematically identical to sampling crops in your dataset, pushing them through the network and training directly on the individual crops. At the same time, the above training setup is much faster as it avoids redundant computations by using dilated convolutions, see our paper [Deep learning with convolutional neural networks for EEG decoding and visualization](https://arxiv.org/abs/1703.05051). However, the two setups are only mathematically identical in case (1) your network does not use any padding and (2) your loss function leads to the same gradients when using the averaged output. The first is true for our shallow and deep ConvNet models and the second is true for the log-softmax outputs and negative log likelihood loss that is typically used for classification in PyTorch.

Most of the code for cropped decoding is identical to the [Trialwise Decoding Tutorial](Trialwise_Decoding.html), differences are explained in the text.

## Enable logging

In [1]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

## Load data

In [2]:
import mne
from mne.io import concatenate_raws

# 5,6,7,10,13,14 are codes for executed and imagined hands/feet
subject_id = 22
event_codes = [5,6,9,10,13,14]
#event_codes = [3,4,5,6,7,8,9,10,11,12,13,14]

# This will download the files if you don't have them yet,
# and then return the paths to the files.
physionet_paths = mne.datasets.eegbci.load_data(subject_id, event_codes)

# Load each of the files
parts = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto', verbose='WARNING')
         for path in physionet_paths]

# Concatenate them
raw = concatenate_raws(parts)

# Find the events in this dataset
events, _ = mne.events_from_annotations(raw)

# Use only EEG channels
eeg_channel_inds = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# Extract trials, only using EEG channels
epoched = mne.Epochs(raw, events, dict(hands_or_left=2, feet_or_right=3), tmin=1, tmax=4.1, proj=False, picks=eeg_channel_inds,
                baseline=None, preload=True)

## Convert data to Braindecode format

In [3]:
import numpy as np
# Convert data from volt to millivolt
# Pytorch expects float32 for input and int64 for labels.
X = (epoched.get_data() * 1e6).astype(np.float32)
y = (epoched.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1

In [4]:
from braindecode.datautil.signal_target import SignalAndTarget

train_set = SignalAndTarget(X[:40], y=y[:40])
valid_set = SignalAndTarget(X[40:70], y=y[40:70])


## Create the model

<div class="alert alert-info">

As in the trialwise decoding tutorial, we will use the Braindecode model class directly to perform the training in a few lines of code. If you instead want to use your own training loop, have a look at the [Cropped Manual Training Loop Tutorial](./Cropped_Manual_Training_Loop.html).
</div>

For cropped decoding, we now transform the model into a model that outputs a dense time series of predictions.
For this, we manually set the length of the final convolution layer to some length that makes the receptive field of the ConvNet smaller than the number of samples in a trial (see `final_conv_length=12` in the model definition). 

In [6]:
from braindecode.models import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = train_set.X.shape[1]

model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=None,
                        final_conv_length=12)
if cuda:
    model.cuda()


Now we supply `cropped=True` to our compile function 

In [7]:
from braindecode.torch_ext.optimizers import AdamW
import torch.nn.functional as F
#optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
model.compile(loss=F.nll_loss, optimizer=optimizer,  iterator_seed=1, cropped=True)

## Run the training

For fitting, we must supply the super crop size. Here, we it to 450 by setting `input_time_length = 450`.

In [8]:
input_time_length = 450
model.fit(train_set.X, train_set.y, n_epochs=30, batch_size=64, scheduler='cosine',
          input_time_length=input_time_length,
         validation_data=(valid_set.X, valid_set.y),)

2020-01-13 17:49:07,369 INFO : Run until first stop...
2020-01-13 17:49:08,967 INFO : Epoch 0
2020-01-13 17:49:08,968 INFO : train_loss                20.30271
2020-01-13 17:49:08,968 INFO : valid_loss                19.24412
2020-01-13 17:49:08,968 INFO : train_misclass            0.52500
2020-01-13 17:49:08,969 INFO : valid_misclass            0.53333
2020-01-13 17:49:08,969 INFO : runtime                   0.00000
2020-01-13 17:49:08,970 INFO : 
2020-01-13 17:49:12,065 INFO : Time only for training updates: 3.09s
2020-01-13 17:49:13,294 INFO : Epoch 1
2020-01-13 17:49:13,295 INFO : train_loss                7.17172
2020-01-13 17:49:13,296 INFO : valid_loss                5.94951
2020-01-13 17:49:13,296 INFO : train_misclass            0.52500
2020-01-13 17:49:13,297 INFO : valid_misclass            0.53333
2020-01-13 17:49:13,297 INFO : runtime                   4.69565
2020-01-13 17:49:13,298 INFO : 
2020-01-13 17:49:16,031 INFO : Time only for training updates: 2.72s
2020-01-13 17

2020-01-13 17:50:19,911 INFO : valid_misclass            0.20000
2020-01-13 17:50:19,911 INFO : runtime                   4.24223
2020-01-13 17:50:19,912 INFO : 
2020-01-13 17:50:22,548 INFO : Time only for training updates: 2.63s
2020-01-13 17:50:23,826 INFO : Epoch 18
2020-01-13 17:50:23,826 INFO : train_loss                0.12788
2020-01-13 17:50:23,827 INFO : valid_loss                0.51953
2020-01-13 17:50:23,827 INFO : train_misclass            0.00000
2020-01-13 17:50:23,828 INFO : valid_misclass            0.16667
2020-01-13 17:50:23,828 INFO : runtime                   3.88180
2020-01-13 17:50:23,829 INFO : 
2020-01-13 17:50:26,551 INFO : Time only for training updates: 2.71s
2020-01-13 17:50:27,809 INFO : Epoch 19
2020-01-13 17:50:27,810 INFO : train_loss                0.11284
2020-01-13 17:50:27,811 INFO : valid_loss                0.50268
2020-01-13 17:50:27,811 INFO : train_misclass            0.00000
2020-01-13 17:50:27,812 INFO : valid_misclass            0.13333
202

In [9]:
model.epochs_df

,train_loss,valid_loss,train_misclass,valid_misclass,runtime
0,20.302706,19.244120,0.525,0.533333,0.000000
1,7.171721,5.949515,0.525,0.533333,4.695646
2,4.326042,3.223247,0.525,0.500000,3.965332
3,2.710786,1.872921,0.500,0.466667,4.092076
4,1.596519,1.097036,0.450,0.400000,4.260404
5,0.997887,0.761809,0.375,0.300000,3.933767
6,0.674779,0.612125,0.300,0.200000,3.994275
7,0.484044,0.548652,0.200,0.133333,3.867371
8,0.380794,0.537480,0.150,0.166667,3.933336
9,0.325519,0.554661,0.125,0.200000,4.193529


Eventually, we arrive at 90% accuracy, so 27 from 30 trials are correctly predicted.

## Evaluation

In [10]:
test_set = SignalAndTarget(X[70:], y=y[70:])

model.evaluate(test_set.X, test_set.y)

{'loss': 0.46323806047439575,
 'misclass': 0.15000000000000002,
 'runtime': 0.0005068778991699219}

We can now predict entire trials as before or individual crops.

In [11]:
model.predict(test_set.X)

array([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0])

In [12]:
model.predict_outs(test_set.X)

array([[-1.1952435 , -0.54510456],
       [-0.05865435, -3.3187828 ],
       [-1.465686  , -0.7026442 ],
       [-0.09450155, -3.0328774 ],
       [-0.0284515 , -4.385389  ],
       [-0.00693051, -5.2352915 ],
       [-3.5844257 , -0.20425275],
       [-0.39406726, -1.4822389 ],
       [-0.01973766, -4.254665  ],
       [-0.20424142, -4.0091424 ],
       [-0.90883666, -0.6412064 ],
       [-2.8004482 , -0.31948426],
       [-0.694196  , -1.0596985 ],
       [-1.1555578 , -0.49371663],
       [-0.1265099 , -2.686141  ],
       [-1.2716458 , -0.77929556],
       [-2.8155317 , -0.33894047],
       [-1.2942142 , -0.5856274 ],
       [-0.3505226 , -2.0671346 ],
       [-0.63738656, -1.3469466 ]], dtype=float32)

For individual crops, provide `individual_crops=True`. This for example can be useful to plot accuracies over time:

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib
%matplotlib inline
matplotlib.style.use('seaborn')
labels_per_trial_per_crop = model.predict(test_set.X, individual_crops=True)
accs_per_crop = [l == y for l,y in zip(labels_per_trial_per_crop, test_set.y)]
accs_per_crop = np.mean(accs_per_crop, axis=0)
plt.figure(figsize=(8,3))
plt.plot(accs_per_crop * 100)
plt.title("Accuracies per timestep", fontsize=16)
plt.xlabel('Timestep in trial', fontsize=14)
plt.ylabel('Accuracy [%]', fontsize=14)

Raw outputs could be used to visualize a prediction probability across a trial. Here we look at the first trial.

In [ ]:
cropped_outs = model.predict_outs(test_set.X, individual_crops=True)
i_trial = 0
# log-softmax outputs need to be exponentiated to get probabilities
cropped_probs = np.exp(cropped_outs[i_trial])
plt.figure(figsize=(8,3))
plt.plot(cropped_probs.T)
plt.title("Network probabilities for trial {:d} of class {:d}".format(
    i_trial, test_set.y[i_trial]), fontsize=16)
plt.legend(("Class 0", "Class 1"), fontsize=12)
plt.xlabel("Timestep within trial", fontsize=14)
plt.ylabel("Probabilities", fontsize=14)

## Dataset references


 This dataset was created and contributed to PhysioNet by the developers of the [BCI2000](http://www.schalklab.org/research/bci2000) instrumentation system, which they used in making these recordings. The system is described in:
 
     Schalk, G., McFarland, D.J., Hinterberger, T., Birbaumer, N., Wolpaw, J.R. (2004) BCI2000: A General-Purpose Brain-Computer Interface (BCI) System. IEEE TBME 51(6):1034-1043.

[PhysioBank](https://physionet.org/physiobank/) is a large and growing archive of well-characterized digital recordings of physiologic signals and related data for use by the biomedical research community and further described in:

    Goldberger AL, Amaral LAN, Glass L, Hausdorff JM, Ivanov PCh, Mark RG, Mietus JE, Moody GB, Peng C-K, Stanley HE. (2000) PhysioBank, PhysioToolkit, and PhysioNet: Components of a New Research Resource for Complex Physiologic Signals. Circulation 101(23):e215-e220.